In [1]:
import os
import random
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pmdarima as pm 
from sklearn.model_selection import train_test_split
from pmdarima.pipeline import Pipeline

In [2]:
os.getcwd()

'C:\\Users\\niles\\project\\project_work\\notebooks'

In [3]:
# Read data:
#os.chdir("..")
df = pd.read_csv("data.csv")

In [4]:
df.head()

,Time,air_pressure[mmHg],air_temperature[degree celcius],relative_humidity[%],wind_speed[M/S],solar_irridiation[W/m²],total_cloud_cover[from ten],electricity_demand_values[kw],heat_demand_values[kw]
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [5]:
len(df)

70080

In [6]:
df = df.dropna()

In [7]:
len(df)

68792

In [8]:
# Rename Columns:
col_mapping = {"Time": "date",
            "air_pressure[mmHg]": "air_pressure",
            "air_temperature[degree celcius]": "air_temperature",
            "relative_humidity[%]": "relative_humidity",
            "wind_speed[M/S]": "wind_speed",
            "solar_irridiation[W/m²]": "solar_irridiation",
            "total_cloud_cover[from ten]": "total_cloud_cover",
            "electricity_demand_values[kw]": "electricity",
            "heat_demand_values[kw]": "heat"}
df = df.rename(columns = col_mapping)

In [9]:
# Change ts to datetime and set as index:
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d')
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [10]:
len(df)

68792

In [11]:
df.head()

,date,air_pressure,air_temperature,relative_humidity,wind_speed,solar_irridiation,total_cloud_cover,electricity,heat
0,2010-12-01 00:00:00,729.7,25.0,85.0,5.0,0,no clouds,289.567557,432.940036
1,2010-12-01 01:00:00,729.4,27.8,77.0,7.0,0,no clouds,260.168520,473.935901
2,2010-12-01 02:00:00,728.9,33.3,62.0,7.0,0,2/10–3/10.,247.273585,483.278761
3,2010-12-01 03:00:00,731.6,32.2,62.0,2.0,0,5/10.,257.955878,545.921252
4,2010-12-01 04:00:00,732.6,22.8,96.0,3.0,0,2/10–3/10.,258.255081,550.526112


In [12]:
df1 = df[["date",'heat']]

In [13]:
df1 = df1.set_index("date")
df1.head()

,heat
date,
2010-12-01 00:00:00,432.940036
2010-12-01 01:00:00,473.935901
2010-12-01 02:00:00,483.278761
2010-12-01 03:00:00,545.921252
2010-12-01 04:00:00,550.526112


In [14]:
x = df1['heat'].values.reshape(-1, 1) #returns a numpy array
min_max_scaler = StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df1['heat'] = x_scaled

In [15]:
df1.head()

,heat
date,
2010-12-01 00:00:00,0.537153
2010-12-01 01:00:00,0.667290
2010-12-01 02:00:00,0.696948
2010-12-01 03:00:00,0.895801
2010-12-01 04:00:00,0.910419


In [16]:
davg_df = df1.groupby(pd.Grouper(freq='D')).mean()
davg_df.head()

,heat
date,
2010-12-01,0.073040
2010-12-02,-0.212695
2010-12-03,0.291141
2010-12-04,1.031632
2010-12-05,1.426792


In [17]:
len(davg_df)

2920

In [18]:
dftrain, dftest = train_test_split(davg_df, test_size=0.01*7)

In [19]:
len(dftest)

205

In [20]:
dftrain, dfvali = train_test_split(dftrain, test_size=0.2)

In [21]:
len(dftrain)

2172

In [22]:
len(dfvali)

543

In [23]:
# best
random.seed(0)

In [24]:
pipeline = Pipeline([
    ("model", pm.AutoARIMA(test='adf',
                           start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', 
                           suppress_warnings=True, random_state=20, n_fits=100))
])

'''
pipeline = Pipeline([
    ("model",pm.AutoARIMA(test='adf',
                           max_p=3, max_q=3, m=7,
                           start_P=1,start_Q=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=False))
])
'''
pipeline.fit(dftrain)

Performing stepwise search to minimize aic
 ARIMA(3,1,3)(0,1,0)[12]             : AIC=inf, Time=3.22 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=8799.832, Time=0.06 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=7676.675, Time=0.29 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=2.10 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=8253.812, Time=0.09 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=7435.948, Time=0.56 sec
 ARIMA(1,1,0)(3,1,0)[12]             : AIC=7266.982, Time=1.27 sec
 ARIMA(1,1,0)(4,1,0)[12]             : AIC=7205.552, Time=2.06 sec
 ARIMA(1,1,0)(5,1,0)[12]             : AIC=7166.178, Time=4.51 sec
 ARIMA(1,1,0)(5,1,1)[12]             : AIC=inf, Time=38.10 sec
 ARIMA(1,1,0)(4,1,1)[12]             : AIC=inf, Time=21.51 sec
 ARIMA(0,1,0)(5,1,0)[12]             : AIC=7771.263, Time=2.88 sec
 ARIMA(2,1,0)(5,1,0)[12]             : AIC=6913.871, Time=5.29 sec
 ARIMA(2,1,0)(4,1,0)[12]             : AIC=6962.648, Time=2.58 sec
 ARIMA(2,1,0)(5,1,1)[12]             

Pipeline(steps=[('model',
                 AutoARIMA(D=1, d=1, error_action='warn', m=12, max_P=5,
                           max_Q=10, n_fits=100, random_state=20, start_P=0,
                           start_Q=0, start_p=3, start_q=3, test='adf',
                           trace=True))])

In [ ]:
##### univariate arima model predict electricity_demand_values  
# best
#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=5, start_P=0, start_Q=0, max_P=5, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='aic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, random_state=20, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=0, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)

#Arima_model=pm.auto_arima(dftrain['electricity_demand_values'],start_p=3, start_q=3, max_p=5, max_q=6, start_P=1, start_Q=4, max_P=6, max_Q=10, m=12, stepwise=True, seasonal=True, information_criterion='bic', trace=True, d=1, D=1, error_action='warn', suppress_warnings=True, n_fits=100)


In [25]:
Sample_predictions= pipeline.predict_in_sample(exogenous=None, return_conf_int=False)
Actuals = pd.Series.to_numpy(dftrain['heat'])
Residuals = Actuals-Sample_predictions

C:\ProgramData\Anaconda3\lib\site-packages\pmdarima\arima\arima.py:692: UserWarning: As of version 1.5.0 'typ' is no longer a valid arg for predict. In future versions this will raise a TypeError.
  warnings.warn("As of version 1.5.0 'typ' is no longer a valid "


In [26]:
Residuals

array([-0.12706634, -0.71010807,  0.13081852, ...,  1.98433657,
       -1.32253996, -1.67787797])

In [27]:
forecast, conf = pipeline.predict(n_periods=543, return_conf_int=True)
forecast

array([-0.44888389,  0.20305423,  0.2127401 ,  0.03877459, -0.36919695,
        0.79724173,  0.08745892, -0.22448456, -0.53491077,  0.19057857,
       -0.44173273,  0.03757358, -0.50254126,  0.02240206,  0.2700741 ,
       -0.02407765, -0.11695966,  0.9446849 ,  0.02332796, -0.67282648,
       -0.46617369,  0.26416482, -0.54019215,  0.20804513, -0.4946201 ,
       -0.12834881,  0.31813311,  0.01755705, -0.10217268,  1.18680806,
       -0.03348782, -0.68661543, -0.82675009,  0.48937695, -0.45695351,
        0.38977315, -0.49325702, -0.25934143,  0.12748654,  0.07604719,
       -0.02123345,  1.3102883 , -0.24509813, -0.69550328, -0.84245941,
        0.46535708, -0.72178393,  0.26814513, -0.64385916, -0.19334261,
       -0.13473343, -0.12811969, -0.11573478,  1.11788994, -0.23626579,
       -0.8119883 , -0.87652024,  0.55527281, -0.70550401,  0.15812765,
       -0.73884275, -0.10327744, -0.09629591, -0.11188115, -0.33943988,
        1.14602942, -0.12479757, -0.82683288, -0.892158  ,  0.53

In [28]:
Actuals = pd.Series.to_numpy(dfvali['heat'])
Actuals

array([-8.37174302e-01, -3.80693427e-01,  2.33839402e+00, -8.08187553e-01,
       -6.24402081e-01,  2.02348561e+00, -7.59289462e-01, -8.27518858e-01,
       -6.73934790e-01,  6.15677747e-01, -8.37174302e-01, -7.20205590e-01,
       -2.61241349e-01, -5.21897474e-01, -8.08117711e-01,  2.69130931e-01,
        2.06378587e+00,  2.22195966e+00,  3.08328868e-01,  6.95326943e-01,
       -8.33959644e-01, -3.78708289e-02, -8.36699927e-01,  1.36815906e+00,
       -8.32721871e-01,  1.35521542e+00, -8.19567512e-01, -8.07308236e-01,
        6.08904879e-01, -4.57360988e-01,  4.16980145e-01,  1.39592405e-01,
       -3.15765254e-01,  1.45748163e+00, -8.37174302e-01,  3.60064646e-01,
        5.10793337e-01, -8.37174302e-01,  8.00783975e-01,  1.70979311e-01,
       -7.07514323e-01, -8.37174302e-01, -3.83646046e-01, -3.80693427e-01,
       -3.29059941e-01, -5.96798246e-01,  1.24455129e+00, -8.37174302e-01,
        3.32431006e-02, -5.13606809e-01, -8.37174302e-01,  2.97544516e+00,
        5.98340137e-01, -

In [29]:
mseVali = mean_squared_error(Actuals, forecast)
mseVali

2.5375615713335082

In [ ]:
forecasttrain, _ = pipeline.predict(n_periods=2172, return_conf_int=True)
forecasttrain

In [ ]:
Actualstrain = pd.Series.to_numpy(dftrain['heat'])
Actualstrain

In [ ]:
msetrain = mean_squared_error(Actualstrain, forecasttrain)
msetrain

In [ ]:
forecasttest, conf = pipeline.predict(n_periods=205, return_conf_int=True)
forecasttest

In [ ]:
Actualstest = pd.Series.to_numpy(dftest['heat'])
Actualstest

In [ ]:
msetest = mean_squared_error(Actualstest, forecasttest)
msetest

In [ ]:
rmse = math.sqrt(msetest)
print('RMSE: %f' % rmse)

In [ ]:
mse = mean_squared_error(Actuals, forecast)
mse

In [ ]:
rmse = math.sqrt(mse)
print('RMSE: %f' % rmse)